In [1]:
import cv2
import pandas as pd
import numpy as np
import os
import joblib
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy
from skimage.feature import hog
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import preprocessing
from skimage.filters import sobel
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import warnings

warnings.filterwarnings('ignore')


In [2]:
# Creating array

images = []
labels = []

#Define dataset
Data_path = r"C:/Users/vicky/CP/resized/"

#Define Class labels and corresponding Folder names

class_labels = ['phase1', 'phase2', 'phase3', 'phase4', 'phase5']
class_folders = ['phase1', 'phase2', 'phase3', 'phase4', 'phase5']

#Appending all images-


for i in class_folders:
    count = 0
    for filename in os.listdir(Data_path + i):
        # Read the image file directly in grayscale mode
        img = cv2.imread(Data_path + i + '/' + filename, cv2.IMREAD_GRAYSCALE)

        # Check if the image is valid
        if img is not None:
            images.append(img)
            labels.append(class_folders.index(i))
            count += 1
            if count == 500:
                break
            
    print(i + ' : ' + str(count))



#Importing images in numpy arrey

images = np.array(images)
lables = np.array(labels)

phase1 : 226
phase2 : 244
phase3 : 346
phase4 : 372
phase5 : 243


In [3]:
# Function for extracting HOG features
def hog_feature_extraction(images):
    features = []
    orientations = 9
    pixels_per_cell = (8, 8)
    cells_per_block = (2, 2)
    visualize = False
    normalize = True

    for img in images:
        hog_features = hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block, visualize=visualize)
        features.append(hog_features)

    return features

In [4]:
# Function for extracting GLCM features
def glcm_feature_extraction(images):
    features = []
    distances = [1, 2, 3]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    levels = 256
    symmetric = True
    normed = True

    for img in images:
        glcm = graycomatrix(img, distances=distances, angles=angles, levels=levels, symmetric=symmetric, normed=normed)
        contrast = graycoprops(glcm, 'contrast')
        energy = graycoprops(glcm, 'energy')
        homogeneity = graycoprops(glcm, 'homogeneity')
        correlation = graycoprops(glcm, 'correlation')
        glcm_features = [contrast, energy, homogeneity, correlation]
        features.append(np.ravel(glcm_features))

    return features

In [5]:
hog_features = hog_feature_extraction(images)
glcm_features = glcm_feature_extraction(images)

In [6]:
print(hog_features)

[array([0.08785387, 0.        , 0.12424411, ..., 0.30799237, 0.01171681,
       0.06422975]), array([0.11643165, 0.0130844 , 0.        , ..., 0.0574273 , 0.        ,
       0.        ]), array([0.11643165, 0.0130844 , 0.        , ..., 0.0574273 , 0.        ,
       0.        ]), array([0.        , 0.        , 0.27813714, ..., 0.12003622, 0.19196379,
       0.26117915]), array([0.        , 0.        , 0.27813714, ..., 0.12003622, 0.19196379,
       0.26117915]), array([0.06325929, 0.03839075, 0.02018666, ..., 0.00226152, 0.00624484,
       0.02569893]), array([0.06325929, 0.03839075, 0.02018666, ..., 0.00226152, 0.00624484,
       0.02569893]), array([0.06237105, 0.04775873, 0.05692742, ..., 0.04316893, 0.01974843,
       0.01771054]), array([0.08019098, 0.        , 0.00553206, ..., 0.03602129, 0.03198305,
       0.07030741]), array([0.19116572, 0.08301047, 0.09695556, ..., 0.02227479, 0.00644255,
       0.00597747]), array([0.19116572, 0.08301047, 0.09695556, ..., 0.02227479, 0.0064425

In [7]:
print(glcm_features)


[array([5.91032904e+02, 8.04674264e+02, 7.17676394e+02, 8.69468266e+02,
       8.43973102e+02, 8.04674264e+02, 1.05495560e+03, 8.69468266e+02,
       1.00660002e+03, 1.16184872e+03, 1.33584922e+03, 1.21639730e+03,
       1.52658289e-02, 1.17853034e-02, 1.22728021e-02, 1.19185028e-02,
       1.31052277e-02, 1.17853034e-02, 1.02712131e-02, 1.19185028e-02,
       1.20754875e-02, 9.95397099e-03, 9.35666342e-03, 1.00581229e-02,
       2.47812567e-01, 1.50723269e-01, 1.61758730e-01, 1.61434310e-01,
       1.94011863e-01, 1.50723269e-01, 1.02690560e-01, 1.61434310e-01,
       1.65703420e-01, 9.33286361e-02, 7.46731537e-02, 1.06640399e-01,
       9.16695308e-01, 8.86413691e-01, 8.98716124e-01, 8.77266021e-01,
       8.81007329e-01, 8.86413691e-01, 8.50881928e-01, 8.77266021e-01,
       8.58035238e-01, 8.35691527e-01, 8.10880206e-01, 8.27963759e-01]), array([1.10855254e+02, 2.67005475e+02, 1.86470389e+02, 2.70401261e+02,
       2.63553057e+02, 2.67005475e+02, 4.08598410e+02, 2.70401261e+02,
   

In [8]:
a = np.array(hog_features)
a.shape

(1431, 34596)

In [9]:
b = np.array(glcm_features)
b.shape

(1431, 48)

In [10]:
features = np.concatenate((hog_features, glcm_features), axis=1)

In [11]:
features = np.array(features)

In [12]:
features.shape

(1431, 34644)

In [13]:
np.save('feature.npy', features)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=3)

In [15]:
X_train.shape

(1144, 34644)

In [16]:
len(y_train)

1144

In [17]:
svm_model = SVC(kernel="linear")
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [18]:
import joblib
joblib.dump(svm_model, 'svm_model')

['svm_model']

In [19]:
y_predict = svm_model.predict(X_test)

In [20]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy:", accuracy)

Accuracy: 0.8954703832752613


In [21]:
dt_model = DecisionTreeClassifier()

In [22]:
dt_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [23]:
joblib.dump(dt_model, 'dt_model')

['dt_model']

In [24]:
y_predict = dt_model.predict(X_test)

In [25]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy:", accuracy)

Accuracy: 0.6724738675958188


In [26]:
rf_model = RandomForestClassifier()

In [27]:
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [28]:
joblib.dump(rf_model, 'rf_model')

['rf_model']

In [29]:
y_predict = rf_model.predict(X_test)

In [30]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy:", accuracy)

Accuracy: 0.7700348432055749


In [38]:

knn_model = KNeighborsClassifier(n_neighbors=5)

In [39]:
knn_model.fit(X_train, y_train)

KNeighborsClassifier()

In [40]:
joblib.dump(knn_model, 'knn_model')

['knn_model']

In [41]:
y_predict = knn_model.predict(X_test)

In [42]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy:", accuracy)

Accuracy: 0.5156794425087108


In [43]:
y_predict

array([1, 1, 3, 2, 2, 3, 1, 3, 3, 2, 0, 3, 2, 2, 2, 0, 2, 2, 2, 1, 0, 0,
       3, 2, 1, 1, 2, 3, 3, 3, 1, 0, 3, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 2,
       1, 2, 0, 4, 2, 0, 2, 1, 3, 4, 4, 0, 0, 2, 4, 0, 2, 2, 3, 1, 1, 2,
       4, 2, 0, 0, 0, 1, 0, 2, 2, 4, 2, 3, 0, 3, 3, 3, 3, 0, 3, 0, 2, 3,
       3, 2, 2, 2, 2, 3, 1, 3, 2, 0, 4, 1, 3, 0, 4, 0, 1, 0, 3, 1, 3, 2,
       3, 2, 1, 4, 4, 1, 1, 3, 1, 3, 4, 1, 3, 0, 3, 1, 2, 1, 2, 3, 2, 1,
       3, 2, 2, 1, 2, 0, 2, 4, 4, 1, 3, 1, 4, 3, 0, 3, 2, 1, 1, 2, 4, 3,
       2, 3, 0, 4, 3, 0, 4, 0, 0, 1, 0, 1, 0, 3, 2, 2, 1, 2, 1, 0, 2, 2,
       0, 1, 3, 2, 3, 4, 0, 4, 4, 3, 1, 3, 2, 0, 1, 1, 3, 1, 1, 2, 0, 2,
       0, 4, 3, 2, 0, 2, 2, 0, 1, 4, 2, 2, 1, 1, 3, 3, 2, 0, 1, 0, 1, 2,
       4, 0, 1, 3, 2, 3, 0, 3, 1, 1, 3, 2, 2, 4, 2, 0, 3, 2, 3, 4, 3, 1,
       4, 3, 3, 0, 4, 2, 1, 1, 3, 4, 1, 3, 4, 4, 2, 0, 2, 0, 2, 3, 3, 1,
       0, 0, 2, 3, 0, 2, 0, 3, 2, 0, 2, 4, 3, 0, 3, 1, 1, 0, 2, 2, 2, 3,
       2])

In [44]:
def hog_feature_extraction_testing(img):

    features = []
    orientations = 9
    pixels_per_cell = (8, 8)
    cells_per_block = (2, 2)
    visualize = False
    normalize = True

    hog_features = hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell,
                   cells_per_block=cells_per_block, visualize=visualize)

    features.append(hog_features)

    return features

In [45]:
# Function for extracting GLCM features
def glcm_feature_extraction_testing(img):
    features = []
    distances = [1, 2, 3]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    levels = 256
    symmetric = True
    normed = True


    glcm = graycomatrix(img, distances=distances, angles=angles, levels=levels, symmetric=symmetric, normed=normed)
    contrast = graycoprops(glcm, 'contrast')
    energy = graycoprops(glcm, 'energy')
    homogeneity = graycoprops(glcm, 'homogeneity')
    correlation = graycoprops(glcm, 'correlation')
    glcm_features = [contrast, energy, homogeneity, correlation]
    features.append(np.ravel(glcm_features))

    return features

In [46]:
path = r'C:/Users/vicky/CP/resized/phase1/3.jpg'
img = cv2.imread(path, 0)

cv2.imshow("image", img)
cv2.waitKey(30)
cv2.destroyAllWindows()

In [47]:
a = hog_feature_extraction_testing(img)
print(a)

[array([0.14570219, 0.03107256, 0.07600142, ..., 0.04701809, 0.        ,
       0.        ])]


In [48]:
b = glcm_feature_extraction_testing(img)
print(b)

[array([7.09100184e+01, 3.15963368e+02, 2.78092402e+02, 3.18659623e+02,
       2.04860928e+02, 3.15963368e+02, 6.03115819e+02, 3.18659623e+02,
       3.62547446e+02, 6.22209591e+02, 7.80207402e+02, 7.33765361e+02,
       3.36568147e-02, 2.20574799e-02, 2.39653892e-02, 2.23784824e-02,
       2.65349732e-02, 2.20574799e-02, 1.83328895e-02, 2.23784824e-02,
       2.34811772e-02, 1.70693805e-02, 1.60924141e-02, 1.71200175e-02,
       4.33250886e-01, 2.59017240e-01, 2.92138183e-01, 2.63911626e-01,
       3.25129157e-01, 2.59017240e-01, 1.94420128e-01, 2.63911626e-01,
       2.75163792e-01, 1.70112880e-01, 1.46931983e-01, 1.72163643e-01,
       9.82784304e-01, 9.23286244e-01, 9.32388406e-01, 9.22630299e-01,
       9.50327803e-01, 9.23286244e-01, 8.53381911e-01, 9.22630299e-01,
       9.12205773e-01, 8.49147115e-01, 8.10391058e-01, 8.22091826e-01])]


In [49]:
b_array = np.array(b)  # Convert b to a numpy array
b_reshaped = b_array.reshape(1, -1)  # Reshape b to have one row and the same number of columns as a
features = np.concatenate((a, b_reshaped), axis=1)  # Concatenate the two feature vectors along the second axis

In [50]:
svm_model.predict(features)

array([0])

In [51]:
output = ['phase1', 'phase2', 'phase3', 'phase4', 'phase5']


In [52]:
video_path = r'C:/Users/vicky/Downloads/test01.mp4'
cap = cv2.VideoCapture(video_path)


while True:

    success, image = cap.read()
        
    if not success:
        break
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.resize(gray_image, (256, 256))
    a = hog_feature_extraction_testing(gray_image)
    b = glcm_feature_extraction_testing(gray_image)
    b_array = np.array(b)  # Convert b to a numpy array
    b_reshaped = b_array.reshape(1, -1)  # Reshape b to have one row and the same number of columns as a
    feature = np.concatenate((a, b_reshaped), axis=1)
    
    
    pred = svm_model.predict(feature)
    
    cv2.putText(image, str(output[pred[0]]), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_4)
    cv2.imshow('video', image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        

# release the cap object
cap.release()
# close all windows
cv2.destroyAllWindows()
  
    
    